Downloading IMDB Dataset

In [56]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  49.3M      0  0:00:01  0:00:01 --:--:-- 49.3M


**Preparing the data**

In [57]:
shutil.rmtree('aclImdb/val')

In [58]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_n = base_dir / "train_n"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = 10000
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [60]:
shutil.rmtree('aclImdb/train_n')

# Training Sample Size: 100

In [61]:
for category in ("neg", "pos"):
    os.makedirs(train_n / category)
    files = os.listdir(train_dir / category)
    num_train_samples=100
    train_files = files[-num_train_samples:]
    for fname in train_files:
        shutil.move(train_dir / category / fname,
                    train_n / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train_n", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 200 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Preparing integer sequence datasets**

In [63]:
from tensorflow.keras import layers

max_length = 150
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [64]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, None)]            0         
                                                                 
 tf.one_hot_4 (TFOpLambda)   (None, None, 10000)       0         
                                                                 
 bidirectional_17 (Bidirect  (None, 64)                2568448   
 ional)                                                          
                                                                 
 dropout_17 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2568513 (9.80 MB)
Trainable params: 2568513 (9.80 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

**Training a first basic sequence model**

In [65]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
7/7 [==============================] - 12s 1s/step - loss: 1.8091 - accuracy: 0.5000 - val_loss: 1.0221 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 8s 1s/step - loss: 0.8707 - accuracy: 0.5000 - val_loss: 0.7331 - val_accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 8s 1s/step - loss: 0.7080 - accuracy: 0.6400 - val_loss: 0.7438 - val_accuracy: 0.5000
Epoch 4/10
7/7 [==============================] - 8s 1s/step - loss: 0.6363 - accuracy: 0.6200 - val_loss: 0.7506 - val_accuracy: 0.5000
Epoch 5/10
7/7 [==============================] - 8s 1s/step - loss: 0.5680 - accuracy: 0.7150 - val_loss: 0.6849 - val_accuracy: 0.5288
Epoch 6/10
7/7 [==============================] - 8s 1s/step - loss: 0.5226 - accuracy: 0.7450 - val_loss: 0.7532 - val_accuracy: 0.5000
Epoch 7/10
7/7 [==============================] - 8s 1s/step - loss: 0.4505 - accuracy: 0.8450 - val_loss: 0.6879 - val_accuracy: 0.5199
Epoch 8/10
7/7 [========================

**Instantiating an `Embedding` layer**

In [66]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

**Model that uses an `Embedding` layer trained from scratch**

In [68]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_12 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_19 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_19 (Dropout)        (None, 64)                0         
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [69]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
7/7 [==============================] - 9s 911ms/step - loss: 1.6602 - accuracy: 0.5050 - val_loss: 0.8808 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 5s 787ms/step - loss: 0.7400 - accuracy: 0.5400 - val_loss: 0.7357 - val_accuracy: 0.5069
Epoch 3/10
7/7 [==============================] - 5s 764ms/step - loss: 0.6072 - accuracy: 0.6650 - val_loss: 0.7056 - val_accuracy: 0.5181
Epoch 4/10
7/7 [==============================] - 5s 738ms/step - loss: 0.5148 - accuracy: 0.7650 - val_loss: 0.6833 - val_accuracy: 0.5554
Epoch 5/10
7/7 [==============================] - 4s 723ms/step - loss: 0.4763 - accuracy: 0.8150 - val_loss: 0.7121 - val_accuracy: 0.5469
Epoch 6/10
7/7 [==============================] - 5s 791ms/step - loss: 0.2438 - accuracy: 0.9600 - val_loss: 0.7288 - val_accuracy: 0.5652
Epoch 7/10
7/7 [==============================] - 5s 775ms/step - loss: 0.1281 - accuracy: 0.9850 - val_loss: 0.7296 - val_accuracy: 0.6015
Epoch 8/10
7/7 [====

#### Understanding padding and masking

**Using an `Embedding` layer with masking enabled**

In [70]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_13 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_20 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_20 (Dropout)        (None, 64)                0         
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [71]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
7/7 [==============================] - 16s 1s/step - loss: 1.8803 - accuracy: 0.5000 - val_loss: 1.1719 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 5s 832ms/step - loss: 0.9612 - accuracy: 0.5000 - val_loss: 1.0232 - val_accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 5s 832ms/step - loss: 0.7860 - accuracy: 0.5000 - val_loss: 0.8841 - val_accuracy: 0.5000
Epoch 4/10
7/7 [==============================] - 5s 812ms/step - loss: 0.6177 - accuracy: 0.5400 - val_loss: 0.7503 - val_accuracy: 0.5078
Epoch 5/10
7/7 [==============================] - 5s 824ms/step - loss: 0.5158 - accuracy: 0.7300 - val_loss: 0.7387 - val_accuracy: 0.5120
Epoch 6/10
7/7 [==============================] - 5s 834ms/step - loss: 0.3970 - accuracy: 0.8800 - val_loss: 0.7187 - val_accuracy: 0.5251
Epoch 7/10
7/7 [==============================] - 5s 793ms/step - loss: 0.2462 - accuracy: 0.9750 - val_loss: 0.7244 - val_accuracy: 0.5343
Epoch 8/10
7/7 [======

#### Using pretrained word embeddings

In [13]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2024-05-05 03:11:57--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-05-05 03:11:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-05-05 03:11:57--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

**Parsing the GloVe word-embeddings file**

In [14]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


**Preparing the GloVe word-embeddings matrix**

In [15]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

**Model that uses a pretrained Embedding layer**

In [72]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_10 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_21 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_21 (Dropout)        (None, 64)                0         
                                                                 
 dense_21 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

# Training Sample Size: 800

In [73]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  29.6M      0  0:00:02  0:00:02 --:--:-- 29.6M


In [74]:
shutil.rmtree('aclImdb/val')

In [75]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_n = base_dir / "train_n"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = 10000
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [76]:
shutil.rmtree('aclImdb/train_n')

In [77]:
for category in ("neg", "pos"):
    os.makedirs(train_n / category)
    files = os.listdir(train_dir / category)
    num_train_samples=800
    train_files = files[-num_train_samples:]
    for fname in train_files:
        shutil.move(train_dir / category / fname,
                    train_n / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train_n", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 1600 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [78]:
from tensorflow.keras import layers

max_length = 150
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [80]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, None)]            0         
                                                                 
 tf.one_hot_6 (TFOpLambda)   (None, None, 10000)       0         
                                                                 
 bidirectional_23 (Bidirect  (None, 64)                2568448   
 ional)                                                          
                                                                 
 dropout_23 (Dropout)        (None, 64)                0         
                                                                 
 dense_23 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2568513 (9.80 MB)
Trainable params: 2568513 (9.80 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

**Training a first basic sequence model**

In [81]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
50/50 [==============================] - 13s 200ms/step - loss: 0.9255 - accuracy: 0.5238 - val_loss: 0.6659 - val_accuracy: 0.6430
Epoch 2/10
50/50 [==============================] - 9s 185ms/step - loss: 0.6427 - accuracy: 0.6237 - val_loss: 0.6231 - val_accuracy: 0.6980
Epoch 3/10
50/50 [==============================] - 9s 186ms/step - loss: 0.5569 - accuracy: 0.7437 - val_loss: 0.5746 - val_accuracy: 0.7151
Epoch 4/10
50/50 [==============================] - 9s 183ms/step - loss: 0.4761 - accuracy: 0.8444 - val_loss: 0.7267 - val_accuracy: 0.7451
Epoch 5/10
50/50 [==============================] - 9s 185ms/step - loss: 0.3946 - accuracy: 0.8994 - val_loss: 0.7295 - val_accuracy: 0.7602
Epoch 6/10
50/50 [==============================] - 9s 183ms/step - loss: 0.1670 - accuracy: 0.9494 - val_loss: 0.9272 - val_accuracy: 0.7412
Epoch 7/10
50/50 [==============================] - 9s 182ms/step - loss: 4.8936 - accuracy: 0.6338 - val_loss: 7.4211 - val_accuracy: 0.5182
Epoch

**Model that uses an `Embedding` layer trained from scratch**

In [82]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_14 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_24 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_24 (Dropout)        (None, 64)                0         
                                                                 
 dense_24 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [83]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
50/50 [==============================] - 12s 174ms/step - loss: 0.8631 - accuracy: 0.5188 - val_loss: 0.6805 - val_accuracy: 0.5563
Epoch 2/10
50/50 [==============================] - 7s 137ms/step - loss: 0.6208 - accuracy: 0.6525 - val_loss: 0.6161 - val_accuracy: 0.6616
Epoch 3/10
50/50 [==============================] - 6s 126ms/step - loss: 0.5366 - accuracy: 0.7969 - val_loss: 0.8890 - val_accuracy: 0.6504
Epoch 4/10
50/50 [==============================] - 6s 121ms/step - loss: 0.3111 - accuracy: 0.8956 - val_loss: 1.0224 - val_accuracy: 0.7226
Epoch 5/10
50/50 [==============================] - 6s 122ms/step - loss: 0.1944 - accuracy: 0.9556 - val_loss: 1.6019 - val_accuracy: 0.6681
Epoch 6/10
50/50 [==============================] - 6s 120ms/step - loss: 0.3397 - accuracy: 0.9538 - val_loss: 4.2780 - val_accuracy: 0.6392
Epoch 7/10
50/50 [==============================] - 6s 117ms/step - loss: 0.1624 - accuracy: 0.9769 - val_loss: 2.2742 - val_accuracy: 0.7195
Epoch

**Using an `Embedding` layer with masking enabled**

In [84]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_15 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_25 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_25 (Dropout)        (None, 64)                0         
                                                                 
 dense_25 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [85]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
50/50 [==============================] - 19s 212ms/step - loss: 0.8332 - accuracy: 0.5387 - val_loss: 0.6604 - val_accuracy: 0.5720
Epoch 2/10
50/50 [==============================] - 8s 152ms/step - loss: 0.5950 - accuracy: 0.7506 - val_loss: 0.7695 - val_accuracy: 0.6555
Epoch 3/10
50/50 [==============================] - 7s 141ms/step - loss: 0.2887 - accuracy: 0.9175 - val_loss: 0.8889 - val_accuracy: 0.7430
Epoch 4/10
50/50 [==============================] - 7s 132ms/step - loss: 0.1389 - accuracy: 0.9644 - val_loss: 1.3124 - val_accuracy: 0.7384
Epoch 5/10
50/50 [==============================] - 6s 127ms/step - loss: 0.0664 - accuracy: 0.9794 - val_loss: 1.5764 - val_accuracy: 0.7375
Epoch 6/10
50/50 [==============================] - 6s 127ms/step - loss: 0.0618 - accuracy: 0.9894 - val_loss: 1.9205 - val_accuracy: 0.7311
Epoch 7/10
50/50 [==============================] - 6s 119ms/step - loss: 0.0420 - accuracy: 0.9956 - val_loss: 2.7730 - val_accuracy: 0.7381
Epoch

In [86]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_10 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_26 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_26 (Dropout)        (None, 64)                0         
                                                                 
 dense_26 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

# Training Sample Size: 1600

In [87]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  17.4M      0  0:00:04  0:00:04 --:--:-- 18.8M


In [88]:
shutil.rmtree('aclImdb/val')

In [89]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_n = base_dir / "train_n"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = 10000
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [90]:
shutil.rmtree('aclImdb/train_n')

In [91]:
for category in ("neg", "pos"):
    os.makedirs(train_n / category)
    files = os.listdir(train_dir / category)
    num_train_samples=1600
    train_files = files[-num_train_samples:]
    for fname in train_files:
        shutil.move(train_dir / category / fname,
                    train_n / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train_n", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 3200 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [92]:
from tensorflow.keras import layers

max_length = 150
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [93]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, None)]            0         
                                                                 
 tf.one_hot_7 (TFOpLambda)   (None, None, 10000)       0         
                                                                 
 bidirectional_27 (Bidirect  (None, 64)                2568448   
 ional)                                                          
                                                                 
 dropout_27 (Dropout)        (None, 64)                0         
                                                                 
 dense_27 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2568513 (9.80 MB)
Trainable params: 2568513 (9.80 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

**Training a first basic sequence model**

In [94]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
100/100 [==============================] - 14s 110ms/step - loss: 0.8218 - accuracy: 0.5416 - val_loss: 0.6312 - val_accuracy: 0.7121
Epoch 2/10
100/100 [==============================] - 10s 104ms/step - loss: 0.6078 - accuracy: 0.7041 - val_loss: 0.5103 - val_accuracy: 0.7542
Epoch 3/10
100/100 [==============================] - 10s 102ms/step - loss: 0.5729 - accuracy: 0.8328 - val_loss: 0.7749 - val_accuracy: 0.7243
Epoch 4/10
100/100 [==============================] - 10s 102ms/step - loss: 0.3805 - accuracy: 0.8856 - val_loss: 0.6372 - val_accuracy: 0.7837
Epoch 5/10
100/100 [==============================] - 10s 102ms/step - loss: 0.2338 - accuracy: 0.9422 - val_loss: 0.9340 - val_accuracy: 0.7964
Epoch 6/10
100/100 [==============================] - 10s 102ms/step - loss: 0.1880 - accuracy: 0.9444 - val_loss: 1.2775 - val_accuracy: 0.7919
Epoch 7/10
100/100 [==============================] - 10s 102ms/step - loss: 0.2000 - accuracy: 0.9684 - val_loss: 1.3876 - val_ac

**Model that uses an `Embedding` layer trained from scratch**

In [95]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_16 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_28 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_28 (Dropout)        (None, 64)                0         
                                                                 
 dense_28 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [96]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
100/100 [==============================] - 14s 114ms/step - loss: 0.8118 - accuracy: 0.5638 - val_loss: 0.5774 - val_accuracy: 0.7001
Epoch 2/10
100/100 [==============================] - 9s 85ms/step - loss: 0.6155 - accuracy: 0.7534 - val_loss: 0.5837 - val_accuracy: 0.7446
Epoch 3/10
100/100 [==============================] - 7s 73ms/step - loss: 0.4329 - accuracy: 0.8637 - val_loss: 1.2133 - val_accuracy: 0.7515
Epoch 4/10
100/100 [==============================] - 7s 71ms/step - loss: 0.3596 - accuracy: 0.9109 - val_loss: 1.0515 - val_accuracy: 0.7661
Epoch 5/10
100/100 [==============================] - 6s 62ms/step - loss: 0.2342 - accuracy: 0.9472 - val_loss: 1.4572 - val_accuracy: 0.7738
Epoch 6/10
100/100 [==============================] - 6s 61ms/step - loss: 0.1907 - accuracy: 0.9681 - val_loss: 1.7063 - val_accuracy: 0.7720
Epoch 7/10
100/100 [==============================] - 6s 60ms/step - loss: 0.1384 - accuracy: 0.9797 - val_loss: 2.0417 - val_accuracy: 0.78

**Using an `Embedding` layer with masking enabled**

In [97]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_17 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_29 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_29 (Dropout)        (None, 64)                0         
                                                                 
 dense_29 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [98]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
100/100 [==============================] - 20s 133ms/step - loss: 0.7886 - accuracy: 0.6012 - val_loss: 0.5487 - val_accuracy: 0.7365
Epoch 2/10
100/100 [==============================] - 9s 93ms/step - loss: 0.5285 - accuracy: 0.8059 - val_loss: 0.7809 - val_accuracy: 0.7398
Epoch 3/10
100/100 [==============================] - 8s 82ms/step - loss: 0.3784 - accuracy: 0.9041 - val_loss: 1.6285 - val_accuracy: 0.7845
Epoch 4/10
100/100 [==============================] - 8s 79ms/step - loss: 0.2897 - accuracy: 0.9491 - val_loss: 1.5406 - val_accuracy: 0.7961
Epoch 5/10
100/100 [==============================] - 7s 70ms/step - loss: 0.2180 - accuracy: 0.9613 - val_loss: 1.4738 - val_accuracy: 0.7815
Epoch 6/10
100/100 [==============================] - 7s 66ms/step - loss: 0.1592 - accuracy: 0.9775 - val_loss: 1.8206 - val_accuracy: 0.7868
Epoch 7/10
100/100 [==============================] - 7s 67ms/step - loss: 0.1099 - accuracy: 0.9856 - val_loss: 1.7729 - val_accuracy: 0.77

In [99]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_31 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_10 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_30 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_30 (Dropout)        (None, 64)                0         
                                                                 
 dense_30 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

# Training Sample Size: 2400

In [118]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  42.5M      0  0:00:01  0:00:01 --:--:-- 42.5M


In [119]:
shutil.rmtree('aclImdb/val')

In [120]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_n = base_dir / "train_n"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = 10000
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [121]:
shutil.rmtree('aclImdb/train_n')

In [122]:
for category in ("neg", "pos"):
    os.makedirs(train_n / category)
    files = os.listdir(train_dir / category)
    num_train_samples=2400
    train_files = files[-num_train_samples:]
    for fname in train_files:
        shutil.move(train_dir / category / fname,
                    train_n / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train_n", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 4800 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [105]:
from tensorflow.keras import layers

max_length = 150
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [115]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, None)]            0         
                                                                 
 tf.one_hot_10 (TFOpLambda)  (None, None, 10000)       0         
                                                                 
 bidirectional_36 (Bidirect  (None, 64)                2568448   
 ional)                                                          
                                                                 
 dropout_36 (Dropout)        (None, 64)                0         
                                                                 
 dense_36 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2568513 (9.80 MB)
Trainable params: 2568513 (9.80 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

**Training a first basic sequence model**

In [116]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
150/150 [==============================] - 15s 80ms/step - loss: 0.7865 - accuracy: 0.5508 - val_loss: 0.6253 - val_accuracy: 0.6247
Epoch 2/10
150/150 [==============================] - 11s 76ms/step - loss: 0.6022 - accuracy: 0.7202 - val_loss: 0.5981 - val_accuracy: 0.6761
Epoch 3/10
150/150 [==============================] - 11s 75ms/step - loss: 0.5621 - accuracy: 0.8325 - val_loss: 0.5605 - val_accuracy: 0.7801
Epoch 4/10
150/150 [==============================] - 11s 75ms/step - loss: 0.5077 - accuracy: 0.8392 - val_loss: 4.1745 - val_accuracy: 0.6895
Epoch 5/10
150/150 [==============================] - 11s 74ms/step - loss: 0.4082 - accuracy: 0.8867 - val_loss: 0.6043 - val_accuracy: 0.7845
Epoch 6/10
150/150 [==============================] - 11s 75ms/step - loss: 0.3092 - accuracy: 0.9148 - val_loss: 0.8699 - val_accuracy: 0.7756
Epoch 7/10
150/150 [==============================] - 11s 74ms/step - loss: 0.2727 - accuracy: 0.9456 - val_loss: 1.2370 - val_accuracy:

**Model that uses an `Embedding` layer trained from scratch**

In [108]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_33 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_18 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_32 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_32 (Dropout)        (None, 64)                0         
                                                                 
 dense_32 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [109]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
150/150 [==============================] - 16s 88ms/step - loss: 0.7235 - accuracy: 0.5821 - val_loss: 0.8316 - val_accuracy: 0.6507
Epoch 2/10
150/150 [==============================] - 9s 62ms/step - loss: 0.5940 - accuracy: 0.7902 - val_loss: 0.6608 - val_accuracy: 0.7768
Epoch 3/10
150/150 [==============================] - 8s 56ms/step - loss: 0.4604 - accuracy: 0.8683 - val_loss: 1.4596 - val_accuracy: 0.7707
Epoch 4/10
150/150 [==============================] - 7s 48ms/step - loss: 0.3750 - accuracy: 0.9165 - val_loss: 3.4881 - val_accuracy: 0.6936
Epoch 5/10
150/150 [==============================] - 7s 45ms/step - loss: 0.2657 - accuracy: 0.9463 - val_loss: 1.3386 - val_accuracy: 0.8046
Epoch 6/10
150/150 [==============================] - 7s 46ms/step - loss: 0.2947 - accuracy: 0.9592 - val_loss: 1.6646 - val_accuracy: 0.7803
Epoch 7/10
150/150 [==============================] - 7s 45ms/step - loss: 0.1998 - accuracy: 0.9750 - val_loss: 2.5277 - val_accuracy: 0.768

**Using an `Embedding` layer with masking enabled**

In [110]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()



Model: "model_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_19 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_33 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_33 (Dropout)        (None, 64)                0         
                                                                 
 dense_33 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [111]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
150/150 [==============================] - 24s 105ms/step - loss: 0.7981 - accuracy: 0.5810 - val_loss: 0.6535 - val_accuracy: 0.6593
Epoch 2/10
150/150 [==============================] - 11s 72ms/step - loss: 0.5778 - accuracy: 0.7715 - val_loss: 0.7717 - val_accuracy: 0.7883
Epoch 3/10
150/150 [==============================] - 9s 60ms/step - loss: 0.4565 - accuracy: 0.8481 - val_loss: 0.6678 - val_accuracy: 0.7617
Epoch 4/10
150/150 [==============================] - 8s 56ms/step - loss: 0.3105 - accuracy: 0.9112 - val_loss: 1.1266 - val_accuracy: 0.7893
Epoch 5/10
150/150 [==============================] - 8s 51ms/step - loss: 0.2642 - accuracy: 0.9450 - val_loss: 1.6530 - val_accuracy: 0.7831
Epoch 6/10
150/150 [==============================] - 7s 50ms/step - loss: 0.1855 - accuracy: 0.9669 - val_loss: 1.8064 - val_accuracy: 0.7765
Epoch 7/10
150/150 [==============================] - 8s 50ms/step - loss: 0.1610 - accuracy: 0.9748 - val_loss: 1.9281 - val_accuracy: 0.7

In [123]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True,
                                    monitor="val_loss")
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_39 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_10 (Embedding)    (None, None, 256)         2560000   
                                                                 
 bidirectional_38 (Bidirect  (None, 64)                73984     
 ional)                                                          
                                                                 
 dropout_38 (Dropout)        (None, 64)                0         
                                                                 
 dense_38 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2634049 (10.05 MB)
Trainable params: 2634049 (10.05 MB)
Non-trainable params: 0 (0.00 Byte)
__________________